In [1]:
from  isotools import Transcriptome
import isotools
print (f'This is isotools version {isotools.__version__}')

import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import logging
import pathlib

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


This is isotools version 0.3.5rc4


In [2]:
path='/project/42/pacbio/golong'
date='2022_12'
project='golong_all'
out_path=f'{path}/06-isotools/{project}/results_{date}/01_transcriptome'
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
plot_path=f'{out_path}/plots'
pathlib.Path(plot_path).mkdir(parents=True, exist_ok=True)
table_path=f'{out_path}/tables'
pathlib.Path(table_path).mkdir(parents=True, exist_ok=True)

ref_fn=f'{path}/../references/gencode/gencode.v36.chr_patch_hapl_scaff.annotation_sorted'
genome_fn=f'{path}/../references/gencode/GRCh38.p13.genome.fa'


In [3]:
#prepare the sample table 
samples=pd.read_csv(path+'/scripts/samples/golong_all.csv').set_index('internal_id')
samples=samples.loc[~samples.file.str.contains('ENCODE')]
samples=samples.drop('CLL201')
samples.file=samples.file.str.replace('05c-raw_align_p13', '05d-raw_align_mm2')
sample_mut_af=pd.read_csv(path+'/scripts/samples/golong_isoseq_mutations.csv', sep='\t')
mut=[]
afreq=[]
for idx,row in samples.iterrows():
    af=sample_mut_af.query(f'id == "{row.TSM_ID}"')
    mut.append(','.join([p for p,a in zip(af.protein, af.mut_fraction) if a==a]))
    afreq.append(','.join([str(a) for p,a in zip(af.protein, af.mut_fraction) if a==a]))
samples['SF3B1_mut']=mut 
samples['SF3B1_AF']=afreq

# assign the short "manuscript" sample names
grp_order={c:i for i,c in enumerate('KNCMB')}
groups={gn: [sa for sa,grn in samples.group.items() if grn==gn] for gn in samples.group.unique()}
new_names={}
for gr in ['CLL', "MDS", "K562", "Nalm6"]:
    count=0
    for mut in ['wt','mut']:
        for sa in groups[f'{gr}_{mut}']:
            count+=1
            new_names[sa]=f'{gr[0]}{count:02d}'
#swap=('CLL198', 'CLL222')
#new_names[swap[0]], new_names[swap[1]]=new_names[swap[1]],new_names[swap[0]]

for gr in ['B-cell']:#, 'K562', 'HL-60', 'GM12878']:
    count=0
    for sa in groups[gr]:
        count+=1
        new_names[sa]=f'{gr[0]}{"" if gr== "B-cell" else "E"}{count:02d}'

#isoseq.sample_table['name']=[new_names[sa] for sa in isoseq.samples]
samples=samples.reset_index()
samples.index=[new_names[sa] for sa in samples['internal_id']]
samples.sort_index( key=lambda x: [(grp_order[c[0]], c[1:]) for c in x], inplace=True)


In [ ]:
# alternatively: isoseq=Transcriptome.load(f'{out_path}/{project}_{date}_isotools_sparse.pkl')
isoseq=Transcriptome.from_reference(ref_fn+'.gff3.gz')
isoseq.save_reference(ref_fn+'.isotools_03.pkl')
isoseq.collapse_immune_genes()
#isoseq=Transcriptome('/project/42/pacbio/MDS/06-isotools/MD1)
for sa,params in samples.iterrows():
    if sa not in isoseq.samples:    
        isoseq.add_sample_from_bam(params['file'], sample_name=sa, **{k:v for k,v in params.items() if k not in ['file', 'total_reads']})

        
isoseq.add_qc_metrics(genome_fn) #takes about 2.5h
isoseq.add_orf_prediction(genome_fn)
isoseq.make_index()
illu_fn={row['name']: glob(f'{path}/07-star/gencode_36/*{row.illumina_sample_id}*sortedByCoord.out.bam') for idx,row in isoseq.sample_table.iterrows() if row.illumina_sample_id ==row.illumina_sample_id }
#missing CLL171 (MRS621) and CLL222 (MRS633)
illu_fn={k:v[0] for k,v in illu_fn.items() if v}
isoseq.add_short_read_coverage(illu_fn)
isoseq.add_filter( 'SUBSTANTIAL_1', 'g.coverage.sum() * .01 < g.coverage[:,trid].sum()',context='transcript') # at least 1 % (default 5%)
isoseq.add_filter( "PROTEIN_CODING", 'transcript_type=="protein_coding"', context='reference')

isoseq.save(f'{out_path}/golong_all_{date}_isotools_full.pkl')
for g in isoseq.iter_genes():
    _=g.coverage
    for tr in g.transcripts:
        tr.pop('TSS', None)
        tr.pop('PAS', None)
        tr.pop('strand', None)            
        tr.pop('coverage', None)
        tr.pop('direct_repeat_len', None)
        tr.pop('coverage', None)
isoseq.save(f'{out_path}/{project}_{date}_isotools_sparse.pkl')
print(isoseq.groups())


In [33]:

groups={k:v for k,v in isoseq.groups().items() if k[:3] in ['CLL', 'MDS']}
group_num={grp:i for i,grp in enumerate(['K562_mut','K562_wt','Nalm6_mut', 'Nalm6_wt','CLL_mut','CLL_wt', 'MDS_mut', 'MDS_wt','B-cell'])}

groups['CL_wt']=[sa for gr in ['K562_wt', 'Nalm6_wt'] for sa in isoseq.groups()[gr]]
groups['CL_mut']=[sa for gr in ['K562_mut', 'Nalm6_mut'] for sa in isoseq.groups()[gr]]
groups['B-cell']=isoseq.groups()['B-cell']
group_idx={gn:[i for i,sa in enumerate(isoseq.samples) if sa in grp] for gn,grp in groups.items()}
group_idx['all']=list(range(len(isoseq.samples)))
group_idx['all_wt']=[sa for gr in groups for sa in group_idx[gr] if '_wt' in gr]
group_idx['all_mut']=[sa for gr in groups for sa in group_idx[gr] if '_mut' in gr]

In [ ]:
isoseq.sample_table.to_csv(f'{out_path}/{project}_{date}_samples.csv')

## add protein domains

In [52]:
info={'hg38':'uniprotName'}
for file in glob(f'{path}/../references/Pfam/ucsc_table_uniprot_*.tsv'):
    what=file[:-4].split('_')[-1]
    if what=='hg38':
        continue
    print(what)
    infocol=info.get(what, 'name')
    isoseq.add_annotation_domains(file,category=what, progress_bar=True)


extracellular


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 8513/8513 [00:22<00:00, 385.45domains/s]
INFO:found domains at 127273 transcript loci
/home/lienhard/.local/lib/python3.10/site-packages/isotools/domains.py:227: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  anno = pd.read_csv(annotation, sep='\t')


interest


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67433/67433 [08:35<00:00, 130.91domains/s]
INFO:found domains at 3599358 transcript loci


transmembrane


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 129138/129138 [16:35<00:00, 129.67domains/s]
INFO:found domains at 8837451 transcript loci


structure


/home/lienhard/.local/lib/python3.10/site-packages/isotools/domains.py:227: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  anno = pd.read_csv(annotation, sep='\t')
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 186235/186235 [28:11<00:00, 110.12domains/s]
INFO:found domains at 15056584 transcript loci


repeat


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 20361/20361 [02:19<00:00, 146.17domains/s]
INFO:found domains at 1157381 transcript loci


In [53]:
isoseq.save(f'{out_path}/{project}_{date}_isotools_sparse_anno_domains.pkl')


INFO:saving transcriptome to /project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_isotools_sparse_anno_domains.pkl


In [29]:
from isotools.domains import import_hmmer_models
pfam_models=import_hmmer_models(f'{path}/../references/Pfam/')
isoseq.add_hmmer_domains(domain_models=pfam_models, genome=genome_fn, query='SUBSTANTIAL_1', ref_query="PROTEIN_CODING", min_coverage=5, progress_bar=True)
isoseq.save(f'{out_path}/{project}_{date}_isotools_sparse_domains.pkl')


INFO:extracting protein sequences...
INFO:found 139994 different protein sequences from 201965 coding transcripts.
INFO:aligning 19632 hmmer domain models to protein sequences...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 19632/19632 [4:00:57<00:00,  1.36domains/s]
INFO:found domains at 974291 loci for 67627 reference transcripts and at 1379423 loci for 77658 long read transcripts.
INFO:saving transcriptome to /project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_isotools_sparse_domains.pkl


In [ ]:
isoseq['ENSG00000276464.4'].ref_transcripts[3]

In [ ]:
isoseq.sample_table

In [14]:
groups=isoseq.groups()

## Output table and gtf file

In [34]:
#export table of transcripts
#isoseq.add_filter( 'SUBSTANTIAL', 'g.coverage.sum() * .05 < g.coverage[:,trid].sum()',context='transcript') # at least 5 % (default)
transcript_tab=isoseq.transcript_table( groups=groups,tpm=True,coverage=True,  min_coverage=5, progress_bar=True, query='SUBSTANTIAL and not (NOVEL_TRANSCRIPT and UNSPLICED)')
orf=[]
default_orf=(None, None, {'CDS':0, 'NMD':True})
for idx,row in transcript_tab.iterrows():
    g=isoseq[row.gene_id]
    tr=g.transcripts[row.transcript_nr]
    tr_orf=tr.get('ORF', default_orf)
    orf.append((tr_orf[0], tr_orf[1], tr_orf[2]['CDS'], tr_orf[2]['NMD']))
orf=pd.DataFrame(orf, columns=['ORF_start', 'ORF_end', 'ORF_length', 'ORF_NMD'])
col_idx=list(transcript_tab.columns).index('novelty_subclasses')+1
for i, col in enumerate(orf.columns):
    transcript_tab.insert(col_idx+i ,col, orf[col])
transcript_tab.to_csv(f'{out_path}/{project}_{date}_substantial_transcripts.csv.gz', index=False, sep='\t')
transcript_tab
ntr=len(transcript_tab)
ngene=len(set(transcript_tab.gene_id))
novel_g=transcript_tab.gene_id.str[:8]=='PB_novel'

n_novel=len(set(transcript_tab.gene_id.loc[novel_g]))
print(f' {ngene} genes with {ntr} transcripts')
print(f' {n_novel} novel genes with {sum(novel_g)} transcripts')
novel_cat=transcript_tab.loc[~novel_g].novelty_class.value_counts().to_dict()
print(f' {ngene - n_novel} known genes have {novel_cat} transcripts')
# 17668 genes with 89659 transcripts
# 1073 novel genes with 2109 transcripts
# 16595 known genes have {'NIC': 32181, 'FSM': 28261, 'NNC': 19857, 'ISM': 6130, 'NOVEL': 1121} transcripts
# -> 1121+2109=3230 novel transcripts


(transcript_tab.groupby('gene_id').size()).value_counts() # transcripts per gene


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 149009/149009 [00:26<00:00, 5718.31genes/s]


 17668 genes with 89659 transcripts
 1073 novel genes with 2109 transcripts
 16595 known genes have {'NIC': 32181, 'FSM': 28261, 'NNC': 19857, 'ISM': 6130, 'NOVEL': 1121} transcripts


1     4737
2     2059
3     1537
4     1369
5     1299
6     1150
7      987
8      879
9      773
10     677
11     552
12     474
13     335
14     235
15     177
16     147
17      92
18      78
19      44
20      29
21      15
23       8
22       6
24       5
27       2
25       1
28       1
dtype: int64

In [35]:
isoseq.write_gtf(f'{out_path}/{project}_{date}_substantial_transcripts.gtf.gz', source='isoseq', min_coverage=5,  gzip=True, query='SUBSTANTIAL and not (NOVEL_TRANSCRIPT and UNSPLICED)')


INFO:writing gzip compressed gtf file to /project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_substantial_transcripts.gtf.gz


In [36]:
transcript_tab_filter=isoseq.transcript_table(samples=isoseq.samples, groups=groups,tpm=True,  min_coverage=10, query='not (RTTS or INTERNAL_PRIMING)', progress_bar=True)
transcript_tab_filter.index=[f'{g}_{nr}' for g,nr in zip(transcript_tab_filter.gene_id, transcript_tab_filter.transcript_nr)]
#transcript_tab_substantial.to_csv(f'{out_path}/{project}_{date}_filtered_transcripts.csv', index=False, sep='\t')
#transcript_tab_substantial.head()
transcript_tab_ncol=len(transcript_tab_filter.columns)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 149009/149009 [00:16<00:00, 9101.61genes/s]


In [38]:
isoseq.write_gtf(f'{out_path}/{project}_{date}_filtered_transcripts.gtf', source='isoseq', min_coverage=10, query='not (RTTS or INTERNAL_PRIMING)', gzip=False)


INFO:writing gtf file to /project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_filtered_transcripts.gtf


In [39]:
#todo: transcripts involved in differential events? (in 04_differential_splicing)

In [40]:
gene_tab=isoseq.gene_table()
gene_tab.to_csv(f'{out_path}/{project}_{date}_all_genes.csv')

In [42]:
gene_counts={g.id:g.coverage.sum(1) for g in isoseq}
gene_counts_df=pd.DataFrame(gene_counts.values(), index=gene_counts.keys(), columns=isoseq.samples)
gene_counts_df.to_csv(f'{out_path}/{project}_{date}_all_genes_counts.csv')

# Extract introns

In [8]:
#get all introns with at least 100 reads in k562
th=10
make_introns=True
if make_introns:
    grp=isoseq.groups()
    introns=[]
    sa=[i for i, sa in enumerate(isoseq.samples) if sa in grp['K562_wt']+grp['K562_mut']]
    for g in isoseq.iter_genes(progress_bar=True):
        cov=g.coverage[sa,:]
        if cov.sum()>th:
            sg=g.segment_graph
            for ni, n in enumerate(sg):
                suc={}
                for trid,nj in n.suc.items():
                    if n.end < sg[nj].start:
                        suc.setdefault(nj,[]).append(trid)
                for nj,trids in suc.items():
                    if cov[:,trids].sum()>th: 
                        # print(ni,nj)
                        introns.append((g.name, g.chrom, g.strand,n.end, sg[nj].start))

    introns=pd.DataFrame(introns,columns=['name', 'chrom', 'strand','start','end'])
    introns.to_csv(f'{out_path}/{project}_{date}_K562_introns_{th}.csv')
    print(f"{len(introns)} 3'AS >{th} reads in K562 samples")
# 45964 3'AS >100 reads in K562 samples
# 106049 3'AS >10 reads in K562 samples

100%|██████████| 149009/149009 [04:25<00:00, 561.32genes/s]


106049 3'AS >10 reads in K562 samples


In [11]:
#get all 3' Splice sites from reference
make_introns=True
if make_introns:
    introns=[]
    for g in isoseq.iter_genes(progress_bar=True, query='not NOVEL_GENE'):
        sg=g.ref_segment_graph
        for ni, n in enumerate(sg):
            suc={}
            for trid,nj in n.suc.items():
                if n.end < sg[nj].start:
                    suc.setdefault(nj,0)
                    suc[nj]+=1
            for nj,_ in suc.items():
                introns.append((g.name, g.chrom, g.strand,n.end, sg[nj].start))

    introns=pd.DataFrame(introns,columns=['name', 'chrom', 'strand','start','end'])
    introns.to_csv(f'{out_path}/{project}_{date}_reference_introns.csv')
    print(f"{len(introns)} 3'AS sites in reference")

100%|██████████| 149009/149009 [00:08<00:00, 17317.93genes/s]


432405 3'AS sites in reference


In [13]:
total_len=0
n_genes=0
for g in isoseq:
    if g.coverage[sa,:].sum()>10:
    #if g.coverage.sum()>10:
        n_genes+=1
        total_len+=(g.end-g.start)
    

In [14]:
print(f"total length of gene bodis of {n_genes} expressed genes: {total_len}")


total length of gene bodis of 10513 expressed genes: 692940528


## Extract AS events for rmats

In [5]:
isoseq.export_alternative_splicing(f'{out_path}/all_events_rMATS/', out_format='mats', reference=False, min_total=100, min_alt_fraction=0.1, progress_bar=True)

100%|██████████| 149009/149009 [18:50<00:00, 131.79genes/s]


In [15]:
isoseq.export_alternative_splicing(f'{out_path}/gencode_rMATS/', out_format='mats', reference=True, progress_bar=True)

100%|██████████| 149009/149009 [00:20<00:00, 7106.12genes/s]


## Comprehensive rMats junction annotation
* junctions
    * read GENCODE junction and isotools junction
    * compute union
* gtf:
    * make gtf without FSM
    * concatenate to GENCODE gtf


SE
imported 49486 events from GENCODE and 579081 events from rMATS
merged  579175 events (99.81% IoMinor)
+: merged  292936 events from [25902,292886] (99.81% IoMinor)
-: merged  286239 events from [23584,286195] (99.81% IoMinor)
RI
imported 29342 events from GENCODE and 9866 events from rMATS
merged  30647 events (86.77% IoMinor)
+: merged  15701 events from [14991,5029] (85.88% IoMinor)
-: merged  14946 events from [14351,4837] (87.70% IoMinor)
MXE
imported 8888 events from GENCODE and 172679 events from rMATS
merged  177674 events (43.80% IoMinor)
+: merged  89777 events from [4611,87263] (45.48% IoMinor)
-: merged  87897 events from [4277,85416] (41.99% IoMinor)
A3SS
imported 21213 events from GENCODE and 25125 events from rMATS
merged  34555 events (55.55% IoMinor)
+: merged  17741 events from [10860,12947] (55.86% IoMinor)
-: merged  16814 events from [10353,12178] (55.22% IoMinor)
A5SS
imported 18089 events from GENCODE and 17291 events from rMATS
merged  27958 events (42.92% Io

,GeneID,geneSymbol,chr,strand,longExonStart_0base,longExonEnd,shortES,shortEE,flankingES,flankingEE
1081,ENSG00000161547.17,SRSF2,chr17,-,76736798,76737331,76736834,76737331,76736153,76736464
